In [37]:
import requests,os,json
import pandas as pd
from bs4 import BeautifulSoup
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
headers = {
    'referer': 'https://www.bloomberg.com/oddlots',
    'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1'
}

def fetchOddLots(url,title):
    docs = requests.get(url,headers=headers).content
    doc = BeautifulSoup(docs)
    doc1 = BeautifulSoup(json.loads(doc.find('script',{'data-component-props':"ArticleBody"}).string)['story']['body'])
    doc1.find(class_="thirdparty-embed__container").decompose()
    html = '''
    <!DOCTYPE html>
    <html>

    <head>
        <meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name=viewport>
        <meta charset=utf-8>
        <meta name="referrer" content="no-referrer">
        <link rel="stylesheet" href="./init.css">
        <title>{title}</title>
    </head>

    <body>
        <div class="BODY">
            <div class="REPLY_LI">
                <h2>{title}</h2>
    '''.format(title=title)
    html = html+str(doc1)+'</body></html>'

    df_odd = pd.DataFrame(columns=['date','title','link','content'])
    date = url.split('/')[5]
    df_odd = df_odd.append({'date':date,'title':title,'link':url,'content':html},ignore_index=True)

    return df_odd

def genHTML(df_odd):

    df_odd = df_odd.sort_values('date',ascending=False)
    
    item = ''
    for idx in df_odd.index:
        date = df_odd.loc[idx,'date']
        link = df_odd.loc[idx,'link']
        html = df_odd.loc[idx,'content']
        title = df_odd.loc[idx,'title']
        name = link.split('/')[-1].split('?')[0]
        item = item+"""<ul class="LI"><li><a class="title" href="./html/{name}.html">[{date}] {title}</a></li></ul>""".format(date=date,name=name,title=title)
        with open('./html/{}.html'.format(name),'w',encoding='utf8') as f:
            f.write(html)

    indexHTML = """    
        <!DOCTYPE html><html><head><meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name=viewport>
        <meta charset=utf-8><link rel="stylesheet" href="./html/init.css">
        <style>.LI li {list-style-type: disc;}
            .LI li {margin-bottom: 8px;}
            .LI a {text-decoration: none;}
            .LI .title {color: #000;}
            .LI .title:hover {color: #f40;}
            .LI .date {font-size: 12px;color: #999;float: right;}
        </style>
        <title>Odd Lots</title>
    </head>

    <body>
        <div class="BODY">
            <h1>Odd Lots</h1>
            <br><br>
        """
    indexHTML = indexHTML+item+"</div></body></html>"
    with open('./index.html','w',encoding='utf8') as f:
        f.write(indexHTML)

df_odd = pd.read_pickle('oddlots.pkl')
doc = requests.get('https://www.bloomberg.com/lineup/api/lazy_load_paginated_module?id=more_articles_list&offset=0&page=oddlots&zone=switch',headers=headers).content
link_title = list(set([(a['href'], a.text) for a in BeautifulSoup(json.loads(doc)['html']).findAll('a') if 'transcript' in a['href'] if '\n\n' not in a.text]))
for link, title in link_title:
    if len(df_odd.loc[df_odd.title.str.contains(title)]) == 0:
        df = fetchOddLots('https://www.bloomberg.com'+link, title)
        df_odd = pd.concat([df, df_odd], ignore_index=True)
        df_odd.to_pickle('oddlots.pkl')
genHTML(df_odd)

In [41]:
df.date[0]

'2023-01-05'

In [42]:
df_odd.loc[df_odd.date > '2023-01-01']

,date,title,link,content
0,2023-01-05,Transcript: Flexport’s Ryan Petersen Says a ‘G...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...
1,2023-01-10,Transcript: Karen Levy on Truck Driver Surveil...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...
2,2023-01-12,Transcript: Neil Dutta and Conor Sen on the Ch...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...
3,2023-01-17,Transcript: Ex-Logger Aims to Beat Elon Musk i...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...
4,2023-01-20,Transcript: The 'Widowmaker' Crypto Trade That...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...
5,2023-01-13,Transcript: Why Banks Are Suddenly Borrowing F...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...
6,2023-01-24,Transcript: What the Heck Is Going on With Egg...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...
7,2023-01-26,Transcript: Why Corporate America Still Runs o...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...
8,2023-01-30,Transcript: The Big Shift That’s Finally Pushi...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...
9,2023-02-01,Transcript: Viktor Shvets Declares Victory for...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...


In [4]:
l = ['/news/articles/2023-03-16/transcript-the-regulatory-blunders-behind-the-svb-disaster',
 '/news/articles/2023-03-13/transcript-how-the-federal-reserve-grew-more-powerful-than-anyone-ever-imagined',
 '/news/articles/2023-03-14/transcript-dan-davies-on-what-brought-down-silicon-valley-bank',
 '/news/articles/2023-03-17/transcript-what-the-boom-in-zero-day-options-means-for-stocks',
 '/news/articles/2023-03-20/transcript-where-stress-is-building-in-the-20-trillion-commercial-real-estate-market',
 '/news/articles/2023-03-06/transcript-brian-armstrong-on-the-challenges-facing-crypto',
 '/news/articles/2023-03-09/transcript-companies-are-telling-us-the-real-reason-they-re-still-raising-prices',
 '/news/articles/2023-03-02/transcript-how-empty-land-in-the-arizona-desert-gets-turned-into-homes',
 '/news/articles/2023-03-02/transcript-jan-toporowski-on-why-capitalists-hate-full-employment',
 '/news/articles/2023-02-17/transcript-brian-deese-on-the-legislative-legacy-of-president-biden-s-first-two-years',
 '/news/articles/2023-02-21/transcript-a-former-cia-official-on-one-year-of-russia-s-war-in-ukraine',
 '/news/articles/2023-02-16/transcript-this-is-what-happens-if-the-us-actually-hits-the-debt-ceiling',
 '/news/articles/2023-02-27/transcript-why-we-don-t-build-new-apartments-for-families',
 '/news/articles/2023-02-13/transcript-what-happens-to-the-laid-off-tech-workers',
 '/news/articles/2023-02-14/transcript-fabio-natalucci-on-how-to-think-about-financial-risk-right-now',
 '/news/articles/2023-02-01/transcript-viktor-shvets-declares-victory-for-team-transitory-and-the-soft-landing',
 '/news/articles/2023-02-06/transcript-steve-eisman-on-the-paradigm-shift-happening-in-the-stock-market',
 '/news/articles/2023-01-30/transcript-the-big-shift-that-s-finally-pushing-rent-lower',
 '/news/articles/2023-01-26/transcript-why-corporate-america-still-runs-on-ancient-software-that-breaks',
 '/news/articles/2023-01-24/transcript-what-the-heck-is-going-on-with-egg-prices',
 '/news/articles/2023-01-20/transcript-the-widowmaker-crypto-trade-that-helped-blow-up-an-industry',
 '/news/articles/2023-01-12/transcript-neil-dutta-and-conor-sen-on-the-chances-of-a-us-soft-landing',
 '/news/articles/2023-01-13/transcript-why-banks-are-suddenly-borrowing-from-the-fed-s-discount-window',
 '/news/articles/2023-01-17/transcript-ex-logger-aims-to-beat-elon-musk-in-electric-trucks',
 '/news/articles/2023-01-10/transcript-karen-levy-on-truck-driver-surveillance',
 '/news/articles/2023-01-05/transcript-flexport-s-ryan-petersen-says-a-great-recession-is-here-for-ocean-shipping',
 '/news/articles/2023-01-03/transcript-the-tracy-and-joe-odd-lots-ama-episode',
 '/news/articles/2023-01-19/transcript-isabella-weber-on-a-new-way-to-think-about-inflation',
 '/news/articles/2023-02-23/transcript-why-interest-rates-on-savings-accounts-are-still-so-low']
l.sort()

C:\Users\PC\AppData\Local\Temp/ipykernel_21968/3080095153.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_odd = df_odd.append({'date':date,'title':title,'link':url,'content':html},ignore_index=True)


,date,title,link,content
0,2023-01-03,ama,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...


In [15]:
doc = requests.get('https://www.bloomberg.com'+l[0],headers=headers).content
doc = BeautifulSoup(doc)
doc1 = BeautifulSoup(json.loads(doc.find('script',{'data-component-props':"ArticleBody"}).string)['story']['body'])
doc1

AttributeError: 'NoneType' object has no attribute 'string'

In [14]:
headers

{'referer': 'https://www.bloomberg.com/oddlots',
 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}

In [9]:
doc1

<html><body><div class="inline-newsletter-top"></div><p><strong>On this special episode of the Odd Lots podcast, co-hosts Tracy Alloway and Joe Weisenthal did an AMA episode, where listeners called in and asked questions of the host. They were joined by producer Carmen Rodriguez. This transcript has been lightly edited for clarity</strong>.</p>
<div class="thirdparty-embed">
<div class="thirdparty-embed__container">
<div style="left: 0; width: 100%; height: 180px; position: relative;"><iframe allowfullscreen="" src="https://omny.fm/shows/odd-lots/the-ama-episode-tracy-and-joe-answer-all-your-ques/embed?style=cover" style="top: 0; left: 0; width: 100%; height: 100%; position: absolute; border: 0;"></iframe></div>
</div>
</div><p><strong>Key insights from the pod</strong>:<br/>How <em>Odd Lots</em> was born — 1:42<br/>How <em>Odd Lots</em> chooses its editorial focus — 6:16<br/>Has <em>Odd Lots </em>changed how we see the world? — 11:54<br/>Who are our favorite guests — 17:58<br/>How muc

In [85]:
df_odd

,date,title,link,content
3,2023-01-03,Transcript: The Tracy and Joe Odd Lots AMA Epi...,https://www.bloomberg.com/news/articles/2023-0...,\n <!DOCTYPE html>\n <html>\n\n <head...
2,2022-12-28,Transcript: Why the Price of Plastic Is Crashi...,https://www.bloomberg.com/news/articles/2022-1...,\n <!DOCTYPE html>\n <html>\n\n <head...
1,2022-12-22,Transcript: What a Bakery Can Tell Us About th...,https://www.bloomberg.com/news/articles/2022-1...,\n <!DOCTYPE html>\n <html>\n\n <head...
0,2022-12-19,Transcript: Tim Duy and Jon Turek on Inflation...,https://www.bloomberg.com/news/articles/2022-1...,\n <!DOCTYPE html>\n <html>\n\n <head...
5,2022-12-15,Transcript: Younger and Menand Explain How We ...,https://www.bloomberg.com/news/articles/2022-1...,\n <!DOCTYPE html>\n <html>\n\n <head...
...,...,...,...,...
133,2021-11-22,Transcript: Here's Why It's So Hard to Fix the...,https://www.bloomberg.com/news/articles/2021-1...,\n <!DOCTYPE html>\n <html>\n\n <head...
134,2021-11-16,"Transcript: ASML, the Obscure Powerhouse at th...",https://www.bloomberg.com/news/articles/2021-1...,\n <!DOCTYPE html>\n <html>\n\n <head...
135,2021-11-11,Transcript: The Perfect Storm That Caused Grai...,https://www.bloomberg.com/news/articles/2021-1...,\n <!DOCTYPE html>\n <html>\n\n <head...
136,2021-11-08,Transcript: Stinson Dean on the Lumber Crash T...,https://www.bloomberg.com/news/articles/2021-1...,\n <!DOCTYPE html>\n <html>\n\n <head...
